In [1]:
import numpy as np
import pandas as pd
import sys
import pickle

In [2]:
import keras 
from keras.models import Sequential 
from keras.layers import Dense 
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
import tensorflow as tf

Using TensorFlow backend.


In [3]:
sys.path.append('.')
sys.path.append('..')
sys.path.append('/..')

In [4]:
import extract_vector as ev
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [5]:
def clean_train_data(train_data, train_label):
    n = train_data.shape[0]
    m = train_data.shape[1]
    nan_ll = []
    for i in range(n):
        if(np.isnan(np.sum(train_data[i:i+1, :]))):
            nan_ll.append(i)

    train_data = np.delete(train_data, nan_ll, 0)
    train_label = np.delete(train_label, nan_ll, 0)
    return train_data, train_label

def get_train_labels(train_label):
    for i in train_label:
        if(i[0] > 4.5):
            i[0] = 1
        else:
            i[0] = 0

        if(i[1] > 4.5):
            i[1] = 1
        else:
            i[1] = 0
            
        if(i[2] > 4.5):
            i[2] = 1
        else:
            i[2] = 0
            
        if(i[3] > 4.5):
            i[3] = 1
        else:
            i[3] = 0
    return train_label

## Wavelet Entropy


### Arousal Model


In [6]:
data = pickle.load(open('../../Dataset/EntropyCompleteDataset/'+ 'WaveletData.pkl', 'rb'))

In [7]:
label= pickle.load(open('../../Dataset/UserLabels.pkl', 'rb'))

In [8]:
train_data = data
train_label = label[:, 0:4]
train_data,train_label = clean_train_data(train_data,train_label)
train_label = get_train_labels(train_label)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train_data,train_label[:,0], test_size=0.33, random_state=42)

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train.astype(np.float64))
X_test = scaler.fit_transform(X_test.astype(np.float64))

In [12]:
# arousal_model=Sequential()
# arousal_model.add(Dense(activation="relu", input_dim=14, units=32, kernel_initializer="uniform"))
# arousal_model.add(Dense(activation="relu", units=64, kernel_initializer="uniform"))
# arousal_model.add(Dense(activation="relu", units=128, kernel_initializer="uniform"))
# arousal_model.add(Dense(activation="relu", units=256, kernel_initializer="uniform"))
# arousal_model.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

In [13]:
# arousal_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
# arousal_model.fit(X_train, y_train,batch_size=10,epochs=10)

In [15]:
def arousal_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(14,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')])
    model.compile(loss='binary_crossentropy', 
        optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
model = arousal_model()

In [18]:
print("Checking accuracy on test set...")
acc = model.evaluate(X_test,y_test,batch_size=10)
print("\nAccuracy on test set: " + str(acc[1]))

Checking accuracy on test set...
25/25 [==============================] - 0s 3ms/step - loss: 0.6263 - accuracy: 0.6680

Accuracy on test set: 0.6679999828338623


In [23]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    arousal_model=Sequential()
    arousal_model.add(Dense(activation="relu", input_dim=14, units=32, kernel_initializer="uniform"))
    arousal_model.add(Dense(activation="relu", units=64, kernel_initializer="uniform"))
    arousal_model.add(Dense(activation="relu", units=128, kernel_initializer="uniform"))
    arousal_model.add(Dense(activation="relu", units=256, kernel_initializer="uniform"))
    arousal_model.add(Dense(activation="relu", units=512, kernel_initializer="uniform"))
    arousal_model.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
    arousal_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return arousal_model

classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [10, 20],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)
grid_search = grid_search.fit(X_train, y_train,verbose = 0)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [26]:
print('Best Parameters after tuning: {}'.format(best_parameters))
print('Best Accuracy after tuning: {}'.format(best_accuracy))

Best Parameters after tuning: {'batch_size': 25, 'epochs': 10, 'optimizer': 'adam'}
Best Accuracy after tuning: 0.6732673267326732
